In [54]:
def read_warehouse_and_moves(filename):
    with open(filename, 'r') as f:
        lines = [line.rstrip('\n') for line in f]
    warehouse_lines = []
    moves_lines = []
    empty_line_found = False

    for line in lines:
        if not empty_line_found:
            if line.strip() == '':
                empty_line_found = True
            else:
                warehouse_lines.append(line)
        else:
            moves_lines.append(line)

    moves = ''.join(moves_lines).replace(' ', '')
    return warehouse_lines, moves

def find_robot(warehouse):
    for r, row in enumerate(warehouse):
        for c, ch in enumerate(row):
            if ch == '@':
                return r, c

def direction_vector(move_char):
    if move_char == '^':
        return (-1, 0)
    elif move_char == 'v':
        return (1, 0)
    elif move_char == '<':
        return (0, -1)
    elif move_char == '>':
        return (0, 1)

def print_warehouse(warehouse):
    for row in warehouse:
        print(''.join(row))

def find_box_gps_coordinates(map_lines, box_char):
    coords = []
    for row_idx, line in enumerate(map_lines):
        for col_idx, char in enumerate(line):
            if char == box_char:
                gps_coord = 100 * row_idx + col_idx
                coords.append(gps_coord)
    return coords
input_file = r"C:\Users\91630\Downloads\AOC\AOCday15\AOC15_Input.txt"
warehouse_lines, moves = read_warehouse_and_moves(input_file)

In [56]:
def simulate_moves_part1(warehouse, moves):
    wh = [list(row) for row in warehouse]
    r, c = find_robot(wh)

    for move in moves:
        dr, dc = direction_vector(move)
        nr, nc = r + dr, c + dc
        if not (0 <= nr < len(wh) and 0 <= nc < len(wh[0])) or wh[nr][nc] == '#':
            continue

        if wh[nr][nc] == '.':
            wh[r][c] = '.'
            wh[nr][nc] = '@'
            r, c = nr, nc
        elif wh[nr][nc] == 'O':
            push_positions = []
            cr, cc = nr, nc
            while 0 <= cr < len(wh) and 0 <= cc < len(wh[0]) and wh[cr][cc] == 'O':
                push_positions.append((cr, cc))
                cr += dr
                cc += dc
            if not (0 <= cr < len(wh) and 0 <= cc < len(wh[0])) or wh[cr][cc] in ('#', 'O'):
                continue
            wh[cr][cc] = 'O'
            for i in range(len(push_positions) - 1, 0, -1):
                pr, pc = push_positions[i]
                pr_prev, pc_prev = push_positions[i - 1]
                wh[pr][pc] = 'O'
            first_box_r, first_box_c = push_positions[0]
            wh[first_box_r][first_box_c] = '@'
            wh[r][c] = '.'
            r, c = first_box_r, first_box_c

    return [''.join(row) for row in wh]
final_map_part1 = simulate_moves_part1(warehouse_lines, moves)
print_warehouse(final_map_part1)
box_coords_part1 = find_box_gps_coordinates(final_map_part1, 'O')
sum(box_coords_part1)

##################################################
#..O...OOO..O.OO..OOOOOOOO.#OO...O.OO......OO....#
#O...OO#..#.#O...OOOOOOOO#O.OO#.OOO....OO.O......#
#..OO...OO.OOO...#O..O.........#OO.......O......O#
##..#..OO.......O.......##.........OOO....O.O..O.#
#..O#.OO........#......OOO......#..O.O...#.OOO...#
#O.....O.............O#...O...#....#O.......O.O..#
#...O...O......O#..#..#O.O............#.....#O..O#
#....O...OOOO..##O.....O#O.OOO.....O....#O...O...#
#O......#OOOOOO.....OO.....OO#..#.#O........#.O.O#
#O....O.#O.OO.O.....#O...........O...O.OOO...OOO.#
#OO..O....#OOO...#...O#.O....#.#..#..OOOO.O..OO.##
#..O.O.....OO#.O.O#O#OO..#OOOO......O...O..O.....#
#....O...O##.O.OOOOOO.#..#O..OO.....O..OO#.#O....#
#...#.OO#.O.....OO.OO....OO..O..#.#..O...O.O....##
#..#...OO.O#O....O.#..........OO..OOOO...##O...O##
#.O..O.OO#O....O.....O.....#...O.O#.OO...OO##....#
#O...OO#O......O..O#........OOOO#...O.....OOOOO..#
#.#..OOO...OO.....OO..O......O#OO......#...O.....#
#...OOO.......................#

1552463

In [55]:
def expand_warehouse(warehouse):
    expand_map = {
        '#': '##',
        'O': '[]',
        '.': '..',
        '@': '@.',
    }
    expanded_warehouse = []
    for line in warehouse:
        expanded_line = ''.join(expand_map[ch] for ch in line)
        expanded_warehouse.append(expanded_line)
    return expanded_warehouse

def simulate_moves_part2(warehouse, moves):
    wh = [list(row) for row in warehouse]
    r, c = find_robot(wh)

    for move in moves:
        dr, dc = direction_vector(move)
        nr, nc = r + dr, c + dc
        if not (0 <= nr < len(wh) and 0 <= nc < len(wh[0])) or wh[nr][nc:nc+2] == '##':
            continue

        if wh[nr][nc:nc+2] == '..':
            wh[r][c] = '.'
            wh[nr][nc] = '@'
            r, c = nr, nc
        elif wh[nr][nc:nc+2] == '[]':
            pr, pc = nr, nc
            while pr < len(wh) and pc + 1 < len(wh[0]) and wh[pr][pc:pc+2] == '[]':
                pr += dr
                pc += dc
            if not (0 <= pr < len(wh) and pc + 1 < len(wh[0])) or wh[pr][pc:pc+2] in ('##', '[]'):
                continue
            cr, cc = nr, nc
            while cr != pr or cc != pc:
                wh[cr][cc] = '.'
                wh[cr][cc + 1] = '.'
                cr -= dr
                cc -= dc
                wh[cr][cc] = '['
                wh[cr][cc + 1] = ']'

            wh[nr][nc] = '@'
            r, c = nr, nc

    return [''.join(row) for row in wh]
expanded_warehouse = expand_warehouse(warehouse_lines)
final_map_part2 = simulate_moves_part2(expanded_warehouse, moves)
print_warehouse(final_map_part2)
box_coords_part2 = find_box_gps_coordinates(final_map_part2, '[')
sum(box_coords_part2)

####################################################################################################
##....[]......[][][]....[]....[]....[]..[]..[][][][]..##[][]......[]..[]..............[][]........##
##[]......[][]##....##..##[]......[][][][]......[]##[]....[]##..[]....[][]......[]..[]............##
##....[][]............[][]........##[]......[][]......[]......##[]................[]............[]##
####....##....[][][]....[]..........[][]........####............[][]....[]..........[]..[]....[]..##
##....[]##..[]..[]....[]....[]..##[][][][]......[][]............##[][]....[][]....##..[][][]......##
##[]..........[]..[][][]........[]........[]##..........[]..##........##................[]..[]....##
##......[]........[]..[][]....[]##....##[]..##....[]......................[]##..........##[]....[]##
##........[]........[][]..[]..####..............##........[]..[]..[]..[]..[]....##[]......[]......##
##[]............##....[][]..........[]......[][]..[][][]..##..[]##..##....[][]..........##.

1570076